# Inspection of concrete structures



In [1]:
# import all libraries needed for this notebook
###############################################
import os
import numpy as np
import pandas as pd 
import shutil

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
import autokeras as ak

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


from PIL import Image

'''
# Classes to predict
seven={
                'crack':'crack',
                'nocrack':'nocrack',
                }
'''

"\n# Classes to predict\nseven={\n                'crack':'crack',\n                'nocrack':'nocrack',\n                }\n"

In [3]:

# create an X and Y_cat for modelling
######################################
#

# Get the list of all imagefiles with their subdirectory
mergeinfo=[]
for i in ['CX', 'UX']:
    path = "../../../concrete_inspection_dataset/SDNET2018/X" + "/"+i
    dir_list = os.listdir(path)
    stop=500
    for j in dir_list:
        full=path + "/" + j
        if i == 'CX':
            CRACK=1
        else:
            CRACK=0
        mergeinfo.append([CRACK,full])
        if stop == 0:
            break
        else:
            stop-=1
 
    
dfinfo=pd.DataFrame(mergeinfo, columns=['label','path'])

# encode categorical
#le = LabelEncoder()
#le.fit(dfinfo['dx'])
#dfinfo['label'] = le.transform(dfinfo["dx"]) 
print(dfinfo.head())
#Use the path to read images.
SIZE=16
dfinfo['image'] = dfinfo['path'].map(lambda x: np.asarray(Image.open(x).resize((SIZE,SIZE))))
#Convert dataframe column of images into numpy array
X = np.asarray(dfinfo['image'].tolist())
X = X/255. # Scale values
Y=dfinfo['label'] # Assign labelencoded values to Y (0 to 6)
#Y_cat = to_categorical(Y, num_classes=2) #Convert to categorical ex. 2 -becomes-> [0,0,1,0,0,0,0]
Y_cat = Y
#print(X.shape, Y_cat.shape)
# output : X, Y_cat



   label                                               path
0      1  ../../../concrete_inspection_dataset/SDNET2018...
1      1  ../../../concrete_inspection_dataset/SDNET2018...
2      1  ../../../concrete_inspection_dataset/SDNET2018...
3      1  ../../../concrete_inspection_dataset/SDNET2018...
4      1  ../../../concrete_inspection_dataset/SDNET2018...


In [10]:
Y_cat

0       1
1       1
2       1
3       1
4       1
       ..
997     0
998     0
999     0
1000    0
1001    0
Name: label, Length: 1002, dtype: int64

In [4]:
dfinfo.head()

,label,path,image
0,1,../../../concrete_inspection_dataset/SDNET2018...,"[[[121, 107, 99], [122, 109, 102], [129, 119, ..."
1,1,../../../concrete_inspection_dataset/SDNET2018...,"[[[125, 111, 102], [123, 109, 100], [121, 106,..."
2,1,../../../concrete_inspection_dataset/SDNET2018...,"[[[121, 105, 95], [117, 103, 94], [124, 110, 1..."
3,1,../../../concrete_inspection_dataset/SDNET2018...,"[[[118, 104, 97], [118, 105, 97], [125, 115, 1..."
4,1,../../../concrete_inspection_dataset/SDNET2018...,"[[[190, 175, 154], [189, 174, 155], [199, 186,..."


In [6]:
# let autokeras select a model using the augmented dataset 
#
# for testing/validating the best model we will use small portion 5% test 5% train
##################################################################################
#
# start model selection
#
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_cat, test_size=0.10, random_state=42, shuffle=True)
x_train_auto, x_test_auto, y_train_auto, y_test_auto = train_test_split(X_test, Y_test, test_size=0.5, random_state=42, shuffle=True)
#
# start model selection
# Define classifier for autokeras. Check 25 different models, each model 25 epochs
# 
clf = ak.ImageClassifier(max_trials=5) #MaxTrials - max. number of keras models to try
clf.fit(x_train_auto, y_train_auto, epochs=3)

#Evaluate the classifier on test data

_, acc = clf.evaluate(x_test_auto, y_test_auto)
print("Accuracy = ", (acc * 100.0), "%")

# get the final best performing model
model = clf.export_model()

#Save the best model 
model.save('../pybin/models/activemodel_best_to_train2')

Trial 1 Complete [00h 00m 01s]
val_loss: 0.5647499561309814

Best val_loss So Far: 0.5647499561309814
Total elapsed time: 00h 00m 01s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
resnet            |vanilla           |image_block_1/block_type
True              |True              |image_block_1/normalize
True              |False             |image_block_1/augment
True              |None              |image_block_1/image_augmentation_1/horizontal_flip
True              |None              |image_block_1/image_augmentation_1/vertical_flip
0                 |None              |image_block_1/image_augmentation_1/contrast_factor
0                 |None              |image_block_1/image_augmentation_1/rotation_factor
0.1               |None              |image_block_1/image_augmentation_1/translation_factor
0                 |None              |image_block_1/image_augmentation_1/zoom_factor
False             |None              |image_block_1/res_net_block_1/pr

KeyboardInterrupt: 

In [7]:
x_train_auto 

array([[[[0.83137255, 0.78823529, 0.71764706],
         [0.82352941, 0.78039216, 0.70980392],
         [0.81176471, 0.76078431, 0.69411765],
         ...,
         [0.79215686, 0.74117647, 0.66666667],
         [0.83921569, 0.79607843, 0.7254902 ],
         [0.83137255, 0.78431373, 0.71764706]],

        [[0.80392157, 0.75686275, 0.6745098 ],
         [0.77254902, 0.72156863, 0.64313725],
         [0.82352941, 0.77254902, 0.70196078],
         ...,
         [0.78431373, 0.73333333, 0.6627451 ],
         [0.81568627, 0.76862745, 0.69411765],
         [0.85882353, 0.81176471, 0.74509804]],

        [[0.8       , 0.74117647, 0.6627451 ],
         [0.79215686, 0.73333333, 0.65882353],
         [0.78431373, 0.73333333, 0.65882353],
         ...,
         [0.80392157, 0.75294118, 0.67843137],
         [0.77647059, 0.7254902 , 0.65490196],
         [0.78823529, 0.7372549 , 0.6627451 ]],

        ...,

        [[0.81568627, 0.76470588, 0.69803922],
         [0.78431373, 0.73333333, 0.66666667]

In [8]:
y_train_auto 

807     0
210     1
662     0
885     0
864     0
774     0
307     1
628     0
625     0
653     0
755     0
797     0
718     0
23      1
693     0
70      1
926     0
543     0
54      1
63      1
803     0
96      1
221     1
439     1
382     1
10      1
289     1
694     0
436     1
822     0
209     1
548     0
673     0
837     0
739     0
261     1
174     1
823     0
527     0
716     0
828     0
292     1
1000    0
275     1
549     0
296     1
493     1
940     0
522     0
881     0
Name: label, dtype: int64

In [12]:
# train the best model
######################
#

# Split the data in train/test set 
X_train_augm, X_test_augm, Y_train_augm, Y_test_augm = train_test_split(X_train, Y_train, test_size=0.3, random_state=42, shuffle=True)

# Load the model to train
model = keras.models.load_model('../pybin/models/activemodel_best_to_train')
# Train
model.fit(X_train_augm,Y_train_augm,epochs=50)
# Evaluate
res=model.evaluate(X_test_augm,Y_test_augm)

# Save the model
model.save('../pybin/models/activemodel_best_trained2')


Epoch 1/50
20/20 [==============================] - 1s 13ms/step - loss: 0.5759 - accuracy: 0.7302
Epoch 2/50
20/20 [==============================] - 0s 14ms/step - loss: 0.4504 - accuracy: 0.8317
Epoch 3/50
20/20 [==============================] - 0s 14ms/step - loss: 0.3840 - accuracy: 0.8429
Epoch 4/50
20/20 [==============================] - 0s 13ms/step - loss: 0.3829 - accuracy: 0.8571
Epoch 5/50
20/20 [==============================] - 0s 14ms/step - loss: 0.3748 - accuracy: 0.8667
Epoch 6/50
20/20 [==============================] - 0s 15ms/step - loss: 0.3456 - accuracy: 0.8635
Epoch 7/50
20/20 [==============================] - 0s 14ms/step - loss: 0.3431 - accuracy: 0.8714
Epoch 8/50
20/20 [==============================] - 0s 14ms/step - loss: 0.3193 - accuracy: 0.8762
Epoch 9/50
20/20 [==============================] - 0s 13ms/step - loss: 0.3317 - accuracy: 0.8778
Epoch 10/50
20/20 [==============================] - 0s 14ms/step - loss: 0.3105 - accuracy: 0.8810
Epoch 11/

INFO:tensorflow:Assets written to: ../pybin/models/activemodel_best_trained2\assets


INFO:tensorflow:Assets written to: ../pybin/models/activemodel_best_trained2\assets


In [13]:
# Do a testprediction
###################
#
model = keras.models.load_model('../pybin/models/activemodel_best_trained2')

# If importing an external image you'll have to rescale it (so : divide by 225.)
#The augmented data in X_test_augm is are already rescaled!
img=np.reshape(X_test_augm[10], (-1, SIZE, SIZE, 3)) 
#
y_pred=model.predict(img)
# print result
print(y_pred)


1/1 [==============================] - 0s 109ms/step
[[0.02607458]]


In [14]:
SIZE=16
for pic in dfinfo['path']:
    img = np.asarray(Image.open(pic).resize((SIZE,SIZE)))
    #Convert dataframe column of images into numpy array
    X = np.asarray(img.tolist())
    X = X/255. # Scale values
    Ximg=np.reshape(X, (-1, SIZE, SIZE, 3)) 
    #
    result=model.predict(Ximg)
    # print result
    print(result)
    input()




1/1 [==============================] - 0s 23ms/step
[[0.13565567]]
1/1 [==============================] - 0s 18ms/step
[[0.15657404]]
1/1 [==============================] - 0s 18ms/step
[[0.15602677]]
1/1 [==============================] - 0s 19ms/step
[[0.12185697]]
1/1 [==============================] - 0s 22ms/step
[[0.08984281]]
1/1 [==============================] - 0s 18ms/step
[[0.04567075]]
1/1 [==============================] - 0s 18ms/step
[[0.70418406]]
1/1 [==============================] - 0s 18ms/step
[[0.07752688]]
1/1 [==============================] - 0s 20ms/step
[[0.13549235]]
1/1 [==============================] - 0s 19ms/step
[[0.13476968]]
1/1 [==============================] - 0s 20ms/step
[[0.1291813]]
1/1 [==============================] - 0s 22ms/step
[[0.02770536]]
1/1 [==============================] - 0s 19ms/step
[[0.01773062]]
1/1 [==============================] - 0s 18ms/step
[[0.14905582]]
1/1 [==============================] - 0s 19ms/step
[[0.1484567

KeyboardInterrupt: Interrupted by user

In [15]:
dfinfo['path'][0]

'../../../concrete_inspection_dataset/SDNET2018/P/CP/001-100.jpg'